# New integration method...

In [ ]:
import numpy as np

def euler_method(func, y0, t, params):
    """
    Integrate a system of ODEs using Euler's method.

    Parameters:
        func : function
            A function representing the right-hand side of the system of ODEs.
            It must take three arguments: t (time), y (state), and params (parameters),
            and return the derivative of y as an array.
        y0 : array_like
            The initial condition on the state y at the initial time t0.
        t : array_like
            A sequence of time points for which to solve for y. The initial
            value point should be the first element of this sequence.
        params : array_like
            Parameters required by the ODE function.

    Returns:
        y : ndarray, shape (len(t), len(y0))
            Array containing the value of y for each desired time in t,
            with the initial value y0 in the first row.
    """
    num_eqns = len(y0)
    y = np.zeros((len(t), num_eqns))
    y[0] = y0
    dt = t[1] - t[0]
    for i in range(1, len(t)):
        y[i] = y[i - 1] + dt * func(t[i - 1], y[i - 1], params)
    return y

def system_of_odes(t, y, params):
    """
    Defines the system of ODEs to be integrated.

    Parameters:
        t : float
            The current time.
        y : array_like
            The current state of the system.
        params : array_like
            Parameters required by the ODE function.

    Returns:
        dydt : array_like
            The derivative of the state y at time t.
    """
    dydt = np.zeros_like(y)
    m1, p1, m2, p2, m3, p3 = y
    k1, k2 = params #only 2 ks are parameters to infer
    k3 = 246.96291990024542
    a1 = a2 = a3 = 24.78485282457379
    g1 = g2 = g3 = 0.024884149937163258
    n1 = n2 = n3 = 5
    b1 = b2 = b3 = 33.82307682700831
    dm1 = dm2 = dm3 = 1.143402097500176
    dp1 = dp2 = dp3 = 0.7833664565550977

    dydt[0] = -dm1 * m1 + (a1 / (1 + ((1/k1) * p2) ** n1)) + g1
    dydt[1] = (b1 * m1) - (dp1 * p1)
    dydt[2] = -dm2 * m2 + (a2 / (1 + ((1/k2) * p3) ** n2)) + g2
    dydt[3] = (b2 * m2) - (dp2 * p2)
    dydt[4] = -dm3 * m3 + (a3 / (1 + ((1/k3) * p1) ** n3)) + g3
    dydt[5] = (b3 * m3)-(dp3 * p3)
    
    return dydt

# Initial conditions
true_params = np.array([
    246.96291990024542, 246.96291990024542])
num_timesteps = 1000
num_trajectories = 6
y0 = np.array([0, 1, 0, 3, 0, 2])
t = np.linspace(0, 100, num_timesteps)

# Integrate the system of ODEs using Euler's method
y = euler_method(system_of_odes, y0, t, true_params) #6x1000
y

In [ ]:
#Now if we want to flatten it...
import torch
y_transposed = y.T
concatenated_trajectories = y_transposed.flatten()
yt = torch.tensor(concatenated_trajectories)
yt.shape

## Beware that there are a couple of other things to be done before "prepare_for_sbi" for SNPE (i.e., enable it to handle multiple parameter sets)